In [51]:
# Importing Libraries

from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

In [52]:
# Transforming Images

transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor()
])

In [ ]:
# Loading Dataset

dataset = datasets.ImageFolder("C:/Users/PMLS/Downloads/Currency_Notes", transform = transform)

In [54]:
# Splitting the Dataset into Train and Validation

train_size = int(0.8 * len(dataset))   # 80% for training
val_size = len(dataset) - train_size   # 20% for validation

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

In [55]:
# Loading data in batches

train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=5, shuffle=False)

In [56]:
# Building the Model using CNN

import torch.nn as nn

class CurrencyCNN(nn.Module):
    def __init__(self, num_classes):
        super(CurrencyCNN, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            
            nn.Flatten(),
            nn.Linear(32 * 32 * 32, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        return self.model(x)

In [ ]:
# Training the model

import torch
import torch.optim as optim

model = CurrencyCNN(num_classes=len(dataset.classes))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(30):
    model.train()
    total_loss = 0

    for images, labels in train_loader:
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # Validation step
    model.eval()
    val_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Epoch {epoch+1}: Train Loss = {total_loss:.4f}, Val Loss = {val_loss:.4f}, Accuracy = {accuracy:.2f}%")

In [58]:
# Saving the Model

torch.save(model.state_dict(), "D:/currency_detector.pth")